# 문제 6

[Kaggle 형] train_prob.csv로 문제 failure 예측하는 모델을 만들고, 

test_prob.csv에 대한 failure가 1일 확률 예측하여 다음과 같은 형식의 answer6.csv를 만들어라. 

측정 지표는 AUC(area under of ROC curve)이다. id 는 테스트 케이스의 id 이고, failure에는 failure가 1이 될 확률이다.

id,failure

16115, 0.1

16116, 0.2


**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [2]:
df_train = pd.read_csv('train_prob.csv', index_col='id')
df_test = pd.read_csv('test_prob.csv', index_col='id')
df_ans = pd.read_csv('test_prob_ans.csv', index_col='id') # 실제 시험에서는 존재 x, 자가 채점용

In [3]:
df_train[['na_1', 'na_2']] = df_train[['measurement_3', 'measurement_5']].isna()
df_test[['na_1', 'na_2']] = df_test[['measurement_3', 'measurement_5']].isna()

,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
id,,,,,,,,,,,,,,,,,,,,,
16115,D,76.89,material_7,material_5,6,6,16,2,7,17.427,...,18.241,11.536,14.690,19.622,11.082,16.323,15.633,16.153,15.640,648.974
16116,D,96.13,material_7,material_5,6,6,9,10,5,18.014,...,19.056,10.746,NaN,18.475,10.174,15.710,16.274,12.229,17.204,741.103
16117,D,124.40,material_7,material_5,6,6,10,15,0,18.374,...,20.689,10.240,16.035,18.224,11.005,15.925,16.257,16.338,17.653,NaN
16118,D,175.84,material_7,material_5,6,6,4,8,15,18.056,...,19.442,12.015,18.195,19.410,10.761,16.176,16.926,13.813,13.615,NaN
16119,D,118.70,material_7,material_5,6,6,15,8,5,16.513,...,19.007,12.615,15.775,18.778,9.683,NaN,17.066,14.690,18.798,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21222,D,111.03,material_7,material_5,6,6,7,13,8,17.288,...,18.216,13.988,NaN,21.500,10.596,14.833,18.031,17.688,15.257,689.243
21223,D,175.93,material_7,material_5,6,6,2,5,9,17.755,...,19.265,9.307,16.060,18.213,12.075,15.825,16.138,14.821,17.246,831.063
21224,D,94.40,material_7,material_5,6,6,9,10,2,16.502,...,18.072,12.160,19.512,18.563,12.369,15.738,18.406,15.719,12.468,640.279


In [7]:
df_test['product_code'].value_counts()

D    5112
Name: product_code, dtype: int64

In [8]:
df_train['product_code'].value_counts()

C    5765
E    5343
B    5250
A    5100
Name: product_code, dtype: int64

In [11]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

imp = IterativeImputer(
    estimator = LinearRegression(fit_intercept=True),
    random_state=123
)

X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']

df_train[X_imp] = df_train.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x[X_imp]), index=x.index, columns=x.columns)
)
df_test[X_imp] = df_test.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x[X_imp]), index=x.index, columns=x.columns)
)

In [12]:
X_mean = ['measurement_{}'.format(i) for i in range(10, 17)]
df_train[X_mean] = df_train.groupby('product_code')[X_mean].transform(lambda x: x.fillna(x.mean()))
df_test[X_mean] = df_test.groupby('product_code')[X_mean].transform(lambda x: x.fillna(x.mean()))

In [15]:
df_train['loading'] = df_train['loading'].fillna(df_train['loading'].mean())
df_test['loading'] = df_test['loading'].fillna(df_train['loading'].mean())

In [ ]:
# LR - SFS - ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17', 'na_1']
# LDA 
# PCA n_components = 7
# RF - {'max_depth': 7, 'min_samples_split': 512, 'n_estimators': 15}

# Kaggle형 풀이 단계

Step 1: 검증 방법을 정하고, 검증 루틴을 만듭니다.

Step 2: Baseline 모델을 만듭니다

Step 3: 모델 선택 루틴을 만듭니다.

Step 4: 모델 개선 작업을 합니다.

In [32]:
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression


X_all = df_test.columns.tolist()
gcv = GroupKFold(4)

for train_idx, test_idx in gcv.split(df_train[X_all], df_train['failure'], groups=df_train['product_code']):
    print(df_train.iloc[train_idx]['product_code'].unique(), df_train.iloc[test_idx]['product_code'].unique())
    
X_lr = ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17', 'na_1']
cross_validate(
    LogisticRegression(solver='lbfgs'), df_train[X_lr], df_train['failure'], 
    groups=df_train['product_code'], scoring='roc_auc', cv=gcv, return_train_score=True
)

['A' 'B' 'E'] ['C']
['A' 'B' 'C'] ['E']
['A' 'C' 'E'] ['B']
['B' 'C' 'E'] ['A']


{'fit_time': array([0.07603407, 0.08020544, 0.03870082, 0.05956149]),
 'score_time': array([0.        , 0.01314425, 0.01000309, 0.        ]),
 'test_score': array([0.5827344 , 0.58852714, 0.57957189, 0.58180685]),
 'train_score': array([0.58973664, 0.59171369, 0.58047486, 0.57853698])}